In [1]:
!pip install selenium beautifulsoup4 webdriver_manager transformers torch --quiet

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--mute-audio")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

# Initialize driver
driver = setup_driver()

In [2]:
import time

# Load the YouTube video URL
url = "https://youtu.be/WKijw2dpFDA?si=EQteR6n1XfoCIYuQ"  # Replace VIDEO_ID with the actual video ID
driver.get(url)
time.sleep(5)  # Wait for the page to load

In [3]:
def scrape_title(driver):
    try:
        # Locate the title using the provided structure
        title_element = driver.find_element(By.CSS_SELECTOR, '#title yt-formatted-string')
        title = title_element.text
        return title
    except Exception as e:
        print("Error:", e)
        return None

# Test the modified scrape_title function
title = scrape_title(driver)
print("Title:", title)


Title: HOLY SH*T! India Drops BOMBSHELL On Justin Trudeau


In [4]:
import re  # Ensure re is imported

def scrape_views_date_tags(driver):
    try:
        # Try to expand the description box, if the button is available
        try:
            show_more = driver.find_element(By.CSS_SELECTOR, 'tp-yt-paper-button#expand')
            driver.execute_script("arguments[0].click();", show_more)
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print("No 'Show more' button found or already expanded.")

        # Locate the info section containing views, upload date, and tags
        info_element = driver.find_element(By.CSS_SELECTOR, 'yt-formatted-string#info')
        
        # Extract views count using regex
        views_match = re.search(r'(\d+[.,]?\d*[K|M|B]?) views', info_element.text)
        if views_match:
            views_str = views_match.group(1).replace(',', '').replace('K', '000').replace('M', '000000').replace('B', '000000000')
            views_count = int(float(views_str))
        else:
            views_count = None

        # Extract upload date using adjusted regex to match formats like "2 Nov 2024" or "Nov 3, 2024"
        date_match = re.search(r'(\d{1,2}\s\w+\s\d{4}|\w+\s\d{1,2},\s\d{4})', info_element.text)
        upload_date = date_match.group(1) if date_match else None

        # Extract tags (hashtags) within the info section
        tag_elements = info_element.find_elements(By.CSS_SELECTOR, 'a.yt-simple-endpoint')
        tags = [tag.text for tag in tag_elements]

        return views_count, upload_date, tags
    except Exception as e:
        print("Error scraping views, upload date, or tags:", e)
        return None, None, []

# Test scrape_views_date_tags function
views_count, upload_date, tags = scrape_views_date_tags(driver)
print("Views:", views_count)
print("Upload Date:", upload_date)
print("Tags:", tags)


Views: 178601
Upload Date: 2 Nov 2024
Tags: ['#pierrepoilievre', '#justintrudeau', '#canada']


In [5]:
def scrape_likes(driver):
    try:
        # Locate the like button using its class name and get the aria-label attribute for the like count
        like_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label*="like this video"]')
        
        # Extract the like count from aria-label (e.g., "like this video along with 4,995 other people")
        likes_text = like_button.get_attribute("aria-label")
        likes_match = re.search(r'(\d+[.,]?\d*[K|M|B]?)', likes_text)
        
        if likes_match:
            # Convert shorthand notations (e.g., "K", "M") to full numbers
            likes_str = likes_match.group(1).replace(',', '').replace('K', '000').replace('M', '000000').replace('B', '000000000')
            likes_count = int(float(likes_str))
        else:
            likes_count = None

        # Dislikes are not publicly available, so set it to None
        dislikes_count = None
        
        return likes_count, dislikes_count
    except Exception as e:
        print("Error scraping likes or dislikes:", e)
        return None, None

# Test scrape_likes function
likes_count, dislikes_count = scrape_likes(driver)
print("Likes:", likes_count)
#print("Dislikes:", dislikes_count)  # Will be None since dislikes are hidden


Likes: 5016


In [6]:
import re  # Ensure re is imported for regex operations

def scrape_channel_info(driver):
    try:
        # Locate the channel name
        channel_name_element = driver.find_element(By.CSS_SELECTOR, '#channel-name #text')
        channel_name = channel_name_element.text.strip()

        # Locate the subscriber count and process it
        subscriber_count_element = driver.find_element(By.CSS_SELECTOR, '#owner-sub-count')
        subscriber_count_text = subscriber_count_element.text.strip().replace("subscribers", "").strip()  # Remove "subscribers" word

        # Convert shorthand notation to integer
        if 'K' in subscriber_count_text:
            subscriber_count = int(float(subscriber_count_text.replace('K', '')) * 1000)
        elif 'M' in subscriber_count_text:
            subscriber_count = int(float(subscriber_count_text.replace('M', '')) * 1000000)
        else:
            subscriber_count = int(subscriber_count_text.replace(',', ''))  # Direct conversion if no shorthand

        return channel_name, subscriber_count
    except Exception as e:
        print("Error scraping channel name or subscriber count:", e)
        return None, None

# Test scrape_channel_info function
channel_name, subscriber_count = scrape_channel_info(driver)
print("Channel Name:", channel_name)
print("Subscriber Count:", subscriber_count)


Channel Name: mistersunshinebaby
Subscriber Count: 421000


In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import json
import time

# Set up Chrome options and service
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
service = Service(ChromeDriverManager().install())

# Set up WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

def scrape_transcript(driver):
    try:
        # Expand the description if necessary
        try:
            more_button = driver.find_element(By.XPATH, '//*[@id="expand"]')
            driver.execute_script("arguments[0].click();", more_button)
            time.sleep(1)
        except Exception as e:
            print("No 'More' button found or already expanded.")

        # Click the transcript button
        transcript_button = driver.find_element(By.XPATH, '//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button')
        transcript_button.click()
        time.sleep(2)  # Wait for the transcript to load

        # Initialize an empty string to store the full transcript
        transcript = ""

        # Loop through all transcript segments
        index = 1
        while True:
            try:
                # Locate each transcript segment by constructing the XPath dynamically
                parent_xpath = f'//*[@id="segments-container"]/ytd-transcript-segment-renderer[{index}]'
                child_xpath = f'{parent_xpath}/div/yt-formatted-string'
                
                # Extract the text of each segment
                text_element = driver.find_element(By.XPATH, child_xpath)
                transcript += text_element.text + "\n"
                
                # Move to the next segment
                index += 1
            except Exception as e:
                # Break the loop when no more segments are found
                break

        return transcript
    except Exception as e:
        print("Error scraping transcript:", e)
        return None

# Usage example
video_url = 'https://www.youtube.com/watch?v=WKijw2dpFDA'  # Example YouTube video URL
driver.get(video_url)
time.sleep(5)  # Wait for the page to load fully

# Call the function to scrape the transcript
transcript_text = scrape_transcript(driver)

# Save the transcript to a JSON file
if transcript_text:
    transcript_data = {"transcript": transcript_text}
    with open("transcript.json", "w", encoding="utf-8") as json_file:
        json.dump(transcript_data, json_file, ensure_ascii=False, indent=4)
    print("Transcript saved to transcript.json")

# Close the driver
driver.quit()


Transcript saved to transcript.json


In [7]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

def scrape_comment_count(driver):
    try:
        # Scroll to load the comments section
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)  # Give some time for the comments section to load

        # Wait until the comment count element is present
        comment_count_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//yt-formatted-string[@class="count-text style-scope ytd-comments-header-renderer"]'))
        )

        # Extract the number of comments
        comment_count_text = comment_count_element.text
        comment_count = int(re.sub('[^0-9]', '', comment_count_text))  # Remove non-numeric characters and convert to integer

        return comment_count
    except Exception as e:
        print("Error scraping comment count:", e)
        return None

# Test scrape_comment_count function
comment_count = scrape_comment_count(driver)
print("Number of Comments:", comment_count)


Number of Comments: 1269


In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

# Setup Selenium options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open YouTube video page
video_url = 'https://www.youtube.com/watch?v=WKijw2dpFDA'  # Replace with your YouTube video URL
driver.get(video_url)

# Initial scroll to trigger comment loading
driver.execute_script("window.scrollTo(0, 800);")
time.sleep(3)

# Extended Wait for comments section
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ytd-comment-thread-renderer"))
    )
except:
    print("Comments section did not load within the wait time.")
    driver.quit()

# Scroll function with limited single scroll attempt
def scroll_once(driver):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3)

# Scroll once to load initial batch of comments
scroll_once(driver)

# Function to scrape comments, likes, and reply counts
def scrape_comments(driver, max_comments=3):
    comments_data = []
    comments_elements = driver.find_elements(By.CSS_SELECTOR, "ytd-comment-thread-renderer")

    for comment_element in comments_elements[:max_comments]:
        try:
            # Extract comment text
            comment_text = comment_element.find_element(By.CSS_SELECTOR, "#content-text").text
            
            # Extract like count for the comment
            try:
                likes = comment_element.find_element(By.CSS_SELECTOR, "#vote-count-middle").text
                likes = int(likes.replace('K', '000').replace('M', '000000'))
            except:
                likes = 0

            # Extract reply count if available
            try:
                reply_button = comment_element.find_element(By.CSS_SELECTOR, "#more-replies")
                replies_text = reply_button.get_attribute("aria-label")
                replies_count = int(replies_text.split()[0].replace('K', '000').replace('M', '000000'))
            except:
                replies_count = 0

            # Append data to list
            comments_data.append({
                "comment_text": comment_text,
                "likes": likes,
                "replies": replies_count
            })

        except Exception as e:
            print("Error scraping comment:", e)

    return comments_data

# Scrape the comments
comments = scrape_comments(driver, max_comments=3)

# Save data to JSON file
with open("youtube_comments.json", "w", encoding="utf-8") as f:
    json.dump(comments, f, ensure_ascii=False, indent=4)

# Close the browser
driver.quit()

# Output confirmation
print("Scraping complete. Data saved to youtube_comments.json")


Comments section did not load within the wait time.


MaxRetryError: HTTPConnectionPool(host='localhost', port=54908): Max retries exceeded with url: /session/f63e251cb4b233788c285feb2698c7cd/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x125535010>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [28]:
pip install youtube-comment-downloader --quiet

Note: you may need to restart the kernel to use updated packages.


In [30]:
from itertools import islice
from youtube_comment_downloader import YoutubeCommentDownloader, SORT_BY_POPULAR
import traceback
import csv
import logging

class ScraperComments:
    def __init__(self, url, sort_by=SORT_BY_POPULAR, comment_limit=10):
        # Ensure the full URL is passed directly without modification
        self.url = url
        self.sort_by = sort_by
        self.comment_limit = comment_limit
        self.comments = []

    def get_comments(self):
        """Scrapes comments from a YouTube video URL."""
        try:
            downloader = YoutubeCommentDownloader()
            comments = downloader.get_comments_from_url(self.url, sort_by=self.sort_by)
            comment_data = []
            for comment in islice(comments, self.comment_limit):
                comment_data.append(comment['text'])
            logging.info("Successfully downloaded comments")
            return comment_data
        except Exception as e:
            logging.error("Error in downloading comments", exc_info=True)
            return None

    def save_to_csv(self, comments, file_name='comments.csv'):
        """Saves scraped comments to a CSV file."""
        if not comments:
            logging.warning("No comments to save.")
            return
        try:
            with open(file_name, 'w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(['Comment'])
                for comment in comments:
                    writer.writerow([comment])
            logging.info(f"Comments saved to {file_name}")
        except Exception as e:
            logging.error("Error in saving comments to CSV", exc_info=True)

    def scrape_and_save_comments(self):
        """Main function to scrape and save comments."""
        comments = self.get_comments()
        self.save_to_csv(comments)
        return comments
# Usage Example:
scraper = ScraperComments("https://www.youtube.com/watch?v=WKijw2dpFDA", comment_limit=20)
scraper.scrape_and_save_comments()


['\u200b\u200b\u200b\u200bGo to https://piavpn.com/Sunshine to get 83% off Private Internet Access with 4 months free! 🚀',
 "How much is Jagmeet tied to this? He can't set foot in India, or he'll be arrested. He and trudeau are propping each other up for a reason.",
 'I am so sorry India. We the people are not behind  our current government. We have asked Trudeau repeatedly to leave. Soon, we all hope.',
 'The Butcher of Canada AKA Justin Trudeau is a global embarrassment',
 "As A Indian Living in India let me tell you few facts..\n1. The Sikhs in Canada the Khalistani movement is to separate a State of India Which is Punjab and Make it a Different country \n2.The Khalistani people were behind the A Flight Bomb blast in 1980s\n3.The Sikhs living in India are peaceful and proud to be Indians\n4.The Indian Government since decades have sent various Notices that these are designated Terrorist living in Canada to send them to India but Trudeau Goverment never cooperated with India\n5.Major

In [8]:
from itertools import islice
from youtube_comment_downloader import YoutubeCommentDownloader, SORT_BY_POPULAR
import logging
import json

class ScraperComments:
    def __init__(self, url, sort_by=SORT_BY_POPULAR, comment_limit=10):
        # Ensure the full URL is passed directly without modification
        self.url = url
        self.sort_by = sort_by
        self.comment_limit = comment_limit
        self.comments = []

    def get_comments(self):
        """Scrapes comments from a YouTube video URL."""
        try:
            downloader = YoutubeCommentDownloader()
            comments = downloader.get_comments_from_url(self.url, sort_by=self.sort_by)
            comment_data = []
            for comment in islice(comments, self.comment_limit):
                comment_data.append(comment['text'])
            logging.info("Successfully downloaded comments")
            return comment_data
        except Exception as e:
            logging.error("Error in downloading comments", exc_info=True)
            return None

    def save_to_json(self, comments, file_name='comments.json'):
        """Saves scraped comments to a JSON file."""
        if not comments:
            logging.warning("No comments to save.")
            return
        try:
            with open(file_name, 'w', encoding='utf-8') as file:
                json.dump(comments, file, ensure_ascii=False, indent=4)
            logging.info(f"Comments saved to {file_name}")
        except Exception as e:
            logging.error("Error in saving comments to JSON", exc_info=True)

    def scrape_and_save_comments(self):
        """Main function to scrape and save comments."""
        comments = self.get_comments()
        self.save_to_json(comments)  # Save comments to JSON
        #return comments

# Usage Example:
scraper = ScraperComments("https://www.youtube.com/watch?v=WKijw2dpFDA", comment_limit=21)
scraper.scrape_and_save_comments()
print("Successfully saved at comments.json")

Successfully saved at comments.json
